🎯 Objective:
In this notebook, we experiment with five pre-trained convolutional neural network models to classify fish species from images.

We will:

🔄 Leverage transfer learning using pre-trained ImageNet models

🐟 Fine-tune these models on our custom 11-class fish dataset

📊 Compare validation accuracies and select the best performing model

💾 Save the best model in .h5 format for future use

✅ Task Breakdown
Step	Description
1. Import Libraries	Load all required libraries and Keras applications
2. Load Dataset	Mount Google Drive and prepare ImageDataGenerator for train/val/test
3. Build Model Function	Create a reusable model-building function with a custom head
4. Train 5 Pretrained Models	VGG16, ResNet50, MobileNetV2, EfficientNetB0, InceptionV3
5. Save the Best Model	Automatically save the model with the highest val accuracy
6. (Optional) Evaluate on Test Set	Load the best .h5 model and test it on unseen data

🧪 Models to Experiment With:
✅ VGG16

✅ ResNet50

✅ MobileNetV2

✅ EfficientNetB0

✅ InceptionV3 (requires input size 299×299)

📁 Dataset Path (Mounted from Drive)
bash
Copy
Edit
/content/drive/MyDrive/fish data/data/
├── train/
├── val/
└── test/
Each folder contains subfolders named after the 11 fish classes.

🏁 Expected Output:
Trained models using transfer learning

.h5 file of the best model saved using ModelCheckpoint

Validation accuracy printed for all five models

(Optional) Final test accuracy of the best model

In [8]:
# importing libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2, InceptionV3, EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import os
from google.colab import files
from google.colab import drive

In [1]:
# step 2 mounting the drive and preparing dataset
drive.mount('/content/drive')

# prepare dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_dir = "/content/drive/MyDrive/fish_data/data/train"
val_dir   = "/content/drive/MyDrive/fish_data/data/val"
test_dir  = "/content/drive/MyDrive/fish_data/data/test"

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=True)

val_gen = datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False)

test_gen = datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False)

class_names = list(train_gen.class_indices.keys())
NUM_CLASSES = len(class_names)


Mounted at /content/drive
Found 6225 images belonging to 11 classes.
Found 1092 images belonging to 11 classes.
Found 581 images belonging to 11 classes.


In [3]:
# building the model function
def build_model(base_model, input_shape=(224, 224, 3), num_classes=11):
    base_model.trainable = False  # Freeze base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


In [4]:
# training and evaluating all pre trained models
models = {
    "VGG16": VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    "ResNet50": ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    "MobileNetV2": MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    "EfficientNetB0": EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    "InceptionV3": InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
}

best_model_name = ""
best_val_acc = 0.0

for name, base_model in models.items():
    print(f"\n🔁 Training {name}...")

    # Adjust size for InceptionV3
    if name == "InceptionV3":
        IMG_SIZE = (299, 299)
        train_gen = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)
        val_gen = datagen.flow_from_directory(val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)
    else:
        IMG_SIZE = (224, 224)
        train_gen = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)
        val_gen = datagen.flow_from_directory(val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

    model = build_model(base_model, input_shape=(*IMG_SIZE, 3), num_classes=NUM_CLASSES)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint = ModelCheckpoint(f"{name}_best.h5", monitor='val_accuracy', save_best_only=True)
    history = model.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=[checkpoint])

    val_acc = max(history.history['val_accuracy'])
    print(f"✅ {name} max val accuracy: {val_acc:.2%}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_name = name


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

🔁 Training VGG16...
Found 6225 images belonging to 11 classes.
Found 1092 images belonging to 11 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.2035 - loss: 2.2594

195/195 ━━━━━━━━━━━━━━━━━━━━ 1642s 8s/step - accuracy: 0.2040 - loss: 2.2584 - val_accuracy: 0.5403 - val_loss: 1.7247
Epoch 2/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.4969 - loss: 1.6615

195/195 ━━━━━━━━━━━━━━━━━━━━ 38s 193ms/step - accuracy: 0.4971 - loss: 1.6612 - val_accuracy: 0.7207 - val_loss: 1.4396
Epoch 3/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 39s 198ms/step - accuracy: 0.6156 - loss: 1.4269 - val_accuracy: 0.7170 - val_loss: 1.2518
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.6741 - loss: 1.2316

...

195/195 ━━━━━━━━━━━━━━━━━━━━ 39s 198ms/step - accuracy: 0.9560 - loss: 0.1843 - val_accuracy: 0.9716 - val_loss: 0.1178
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.9696 - loss: 0.1328

195/195 ━━━━━━━━━━━━━━━━━━━━ 41s 208ms/step - accuracy: 0.9696 - loss: 0.1327 - val_accuracy: 0.9817 - val_loss: 0.0913
Epoch 5/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.9757 - loss: 0.1063

195/195 ━━━━━━━━━━━━━━━━━━━━ 45s 232ms/step - accuracy: 0.9757 - loss: 0.1063 - val_accuracy: 0.9872 - val_loss: 0.0741
Epoch 6/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 45s 232ms/step - accuracy: 0.9806 - loss: 0.0855 - val_accuracy: 0.9817 - val_loss: 0.0778
Epoch 7/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 38s 194ms/step - accuracy: 0.9834 - loss: 0.0797 - val_accuracy: 0.9863 - val_loss: 0.0598
Epoch 8/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.9862 - loss: 0.0654

195/195 ━━━━━━━━━━━━━━━━━━━━ 45s 230ms/step - accuracy: 0.9862 - loss: 0.0654 - val_accuracy: 0.9899 - val_loss: 0.0558
Epoch 9/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.9830 - loss: 0.0618

195/195 ━━━━━━━━━━━━━━━━━━━━ 39s 200ms/step - accuracy: 0.9830 - loss: 0.0618 - val_accuracy: 0.9918 - val_loss: 0.0462
Epoch 10/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 41s 209ms/step - accuracy: 0.9850 - loss: 0.0550 - val_accuracy: 0.9918 - val_loss: 0.0455
✅ InceptionV3 max val accuracy: 99.18%


In [6]:
# evaluating the models from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model

# Adjust test image size based on best model
if best_model_name == "InceptionV3":
    test_img_size = (299, 299)
else:
    test_img_size = (224, 224)

# Recreate test generator with matching size
test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=test_img_size,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Load and evaluate
best_model = load_model(f"{best_model_name}_best.h5")
test_loss, test_acc = best_model.evaluate(test_gen)
print(f"📊 Final Test Accuracy of {best_model_name}: {test_acc:.2%}")




Found 581 images belonging to 11 classes.


19/19 ━━━━━━━━━━━━━━━━━━━━ 90s 4s/step - accuracy: 0.9885 - loss: 0.0666
📊 Final Test Accuracy of InceptionV3: 98.97%


In [9]:
# saving the model
print(f"\n🏆 Best model: {best_model_name} with max val accuracy: {best_val_acc:.2%}")


# downloading the best model
files.download(f"{best_model_name}_best.h5")






🏆 Best model: InceptionV3 with max val accuracy: 99.18%


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
from google.colab import files

# List of model filenames you saved
model_files = [
    "VGG16_best.h5",
    "ResNet50_best.h5",
    "MobileNetV2_best.h5",
    "EfficientNetB0_best.h5"

]

# Download each model
for file in model_files:
    try:
        files.download(file)
    except Exception as e:
        print(f"❌ Could not download {file}: {e}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>